In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv")

In [3]:
df.shape

(900000, 11)

In [4]:
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [5]:
len(df["book_id"].unique())

25474

In [6]:
df.drop(["review_id", "review_text", "date_added", "date_updated", "read_at", "started_at", "n_votes", "n_comments"], axis = 1, inplace = True)

In [7]:
df.head()

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,18245960,5
1,8842281e1d1347389f2ab93d60773d4d,16981,3
2,8842281e1d1347389f2ab93d60773d4d,28684704,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,4


In [8]:
df.shape

(900000, 3)

In [9]:
movie_rating_count = (df.groupby(by = ["book_id"])["rating"].count().reset_index().rename(columns = {"rating": "rating_count"})[["book_id", "rating_count"]])
movie_rating_count.head()

,book_id,rating_count
0,1,390
1,2,410
2,3,709
3,5,480
4,6,446


In [10]:
movie_rating_count.describe()

,book_id,rating_count
count,2.547400e+04,25474.000000
mean,1.492047e+07,35.330141
std,9.740023e+06,67.222413
min,1.000000e+00,1.000000
25%,7.155111e+06,10.000000
50%,1.598224e+07,17.000000
75%,2.274974e+07,34.000000
max,3.632868e+07,1734.000000


In [11]:
df_rating_count = df.merge(movie_rating_count, left_on = 'book_id', right_on = 'book_id', how = 'left')

In [12]:
df_rating_count

,user_id,book_id,rating,rating_count
0,8842281e1d1347389f2ab93d60773d4d,18245960,5,20
1,8842281e1d1347389f2ab93d60773d4d,16981,3,54
2,8842281e1d1347389f2ab93d60773d4d,28684704,3,34
3,8842281e1d1347389f2ab93d60773d4d,27161156,0,116
4,8842281e1d1347389f2ab93d60773d4d,25884323,4,9
...,...,...,...,...
899995,b9450d1c1f97f891c392b1105959b56e,11832081,3,19
899996,b9450d1c1f97f891c392b1105959b56e,16095092,3,9
899997,b9450d1c1f97f891c392b1105959b56e,8430896,4,9
899998,b9450d1c1f97f891c392b1105959b56e,12275680,4,39


In [13]:
df_rating_count = df_rating_count[(df_rating_count["rating_count"]>50)]

In [14]:
df_rating_count.shape

(526446, 4)

In [15]:
df = df_rating_count.drop(["rating_count"], axis = 1)

In [16]:
df.head()

,user_id,book_id,rating
1,8842281e1d1347389f2ab93d60773d4d,16981,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
6,8842281e1d1347389f2ab93d60773d4d,22318578,5
8,8842281e1d1347389f2ab93d60773d4d,22551730,4
9,8842281e1d1347389f2ab93d60773d4d,22816087,5


In [17]:
book_features_df=df.pivot_table(index='book_id',columns='user_id',values='rating').fillna(0)
book_features_df.head()

user_id,000a1016fda6008d1edbba720ca00851,000efb30c5236d7437c3cdf4bf3e4dc7,0011e1a9112b3d798702ef5b20bbf35b,0016a8010771c0c00c97f27dfc5cdd22,0019de4561419b7543238e0979f2f33e,00204424763e8233c5f53f0729f2304f,00214d8b0a020837cccf5f41eb563037,00238d8a4c276c47f5d5e242f54a8f28,002a023d3de233b4bd3ec4fc3e9c581a,002e063d40ae0107a59d8f9c1aa7a423,...,ffe007d7696cf95fb6f85bb92bd79739,ffe7c93bb174f1a4990fe36c5b67141c,ffed06f9c41795ef1d77ba3dd3920b39,fff217d58e1f06f8c00226ce4ca1bea0,fff2d524826237f7da79e5ca786c5f48,fff3a250fbc018ad2c2c2d45c86734da,fff7bfd82b89fa347edfe9a82ac0c61b,fffc34d137f5c5c5e1ca1d6f325a4dcf,fffce7dae5ac5e8fb6288d81658ececc,ffff7cafdaf5196383cb2efca08fb6fe
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
book_features_df.shape

(4073, 11906)

In [19]:
from scipy.sparse import csr_matrix
book_features_df_matrix = csr_matrix(book_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
book_features_df_matrix.shape

(4073, 11906)

In [21]:
query_index = np.random.choice(book_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

2377


In [22]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, book_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 15835031:

1: 4325, with distance of 0.8825012816589484:
2: 10866233, with distance of 0.8832308015867304:
3: 9415951, with distance of 0.8906707892430226:
4: 15726915, with distance of 0.8923326065339274:
5: 8470445, with distance of 0.8931448644605879:
